In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [47]:
class ANN:
    def __init__(self, hidden_layer_sizes, activation_functions=None):
        self.M=hidden_layer_sizes
        self.n_M=len(self.M)
        if type(self.M)==np.ndarray:
            self.M=self.M.tolist()
        
        if activation_functions==None:
            self.F=[ANN.ReLU]*self.n_M
            self.d_F=[ANN.d_ReLU]*self.n_M
        elif len(activation_functions[0])==1:
            self.F=activation_functions[0]*self.n_M          
            self.d_F=activation_functions[1]*self.n_M
        else:
            self.F=activation_functions[0]
            self.d_F=activation_functions[1]
            if len(self.d_F)!=len(self.M):
                raise Exception()
        
        #output layer activation--set to linear for now
        self.F=self.F + [lambda x: x]
        
            
    def fit(self, X, Y, eta = 1e-4, epochs = 10, show_curve = False, batch_sz = 0,
            lambda2 = 0, lambda1 = 0, mu = 0, gamma = 1, epsilon = 1e-10):
        X=X.copy()
        Y=Y.copy()
        if len(X.shape)==1:
            X=X.reshape([len(X),1])
        if len(Y.shape)==1:
            Y=Y.reshape([len(Y),1])
        
        self.N=X.shape[0]
        self.Xcenter=X.mean()
        self.Xscale=max(1,X.max()-X.mean(), X.mean()-X.min())
        self.X=(X-self.Xcenter)/self.Xscale
        self.Y=Y
        self.n_W=self.n_M + 1
        self.W={i:np.random.randn(j,k) for i,j,k in zip(range(self.n_W),
                                                         [X.shape[1]]+self.M, 
                                                        self.M+[Y.shape[1]])}
        self.B = {i:np.random.randn(j) for i,j in zip(range(self.n_W),self.M+[Y.shape[1]])}
        self.Z = {i:None for i in range(self.n_W)}
        J=[]
        
        g_W={i:1 for i in range(self.n_W)}
        g_b={i:1 for i in range(self.n_W)}
        if gamma ==1: epsilon=0
            
        v_W={i:0 for i in range(self.n_W)}
        v_b={i:0 for i in range(self.n_W)}
        
        if batch_sz == 0: batch_sz=self.N
        
        for e in range(epochs):
            permute = np.random.permutation(self.N)
            X=self.X[permute,:]
            Y=self.Y[permute,:]
            for idx in range(self.N//batch_sz):
                x_b=X[idx*batch_sz:(idx+1)*batch_sz,:]
                y_b=Y[idx*batch_sz:(idx+1)*batch_sz,:]
                self.forward(x_b)
                                        
                dH = self.Z[self.n_M] - y_b
                for i in range(len(self.Z)-2,-1,-1):
                    dW = np.matmul(self.Z[i].T, dH) + lambda2 * self.W[1+i] + lambda1 * np.sign(self.W[1+i])
                    g_W[1+i]=gamma*g_W[1+i] + (1- gamma) * dW**2
                    v_W[1+i]=mu*v_W[1+i] - eta * dW/np.sqrt(g_W[1+i] + epsilon)
                    self.W[1+i] += v_W[1+i]
                    
                    db = dH.sum(axis = 0) + lambda2 * self.B[1+i] + lambda1 * np.sign(self.B[1+i])
                    g_b[1+i]=gamma*g_b[1+i] + (1- gamma) * db**2
                    v_b[1+i]=mu*v_b[1+i] - eta * db/np.sqrt(g_b[1+i] + epsilon)
                    self.B[1+i] -= v_b[1+i]
    
                    dZ = np.matmul(dH, self.W[1+i].T)
                    dH = dZ *self.d_F[i](self.Z[i])
    
                dW = np.matmul(x_b.T, dH) + lambda2 * self.W[0] + lambda1 * np.sign(self.W[0])
                g_W[0]=gamma*g_W[0] + (1- gamma) * dW**2
                v_W[0]=mu*v_W[0] - eta * dW/np.sqrt(g_W[0] + epsilon)
                self.W[0] += v_W[0]
                db = dH.sum(axis = 0) + lambda2 * self.B[0] + lambda1 * np.sign(self.B[0])
                g_b[0]=gamma*g_b[0] + (1- gamma) * db**2
                v_b[0]=mu*v_b[0] - eta * db/np.sqrt(g_b[0] + epsilon)
                self.B[0] -= v_b[0]
            self.forward(X)
            if show_curve:
                J.append(self.OLS(Y,self.Z[self.n_M]))
                if lambda2 > 0: J[-1] += lambda2 * np.sum([np.sum(self.W[i]**2)+np.sum(self.B[i]**2) for i in self.W])
                if lambda1 > 0: J[-1] += lambda1 * np.sum([np.sum(np.abs(self.W[i]))+np.sum(np.abs(self.B[i])) for i in self.W])
                        
        print("R^2 is {}".format(self.R2(Y,self.Z[self.n_M])))
        if show_curve: plt.plot(J)
    
    def forward(self,X):        
        self.Z[0] = self.F[0](np.matmul(X,self.W[0]) + self.B[0])
        for i in range(self.n_M):
            self.Z[i+1] = self.F[i+1](np.matmul(self.Z[i],self.W[i+1]) + self.B[i+1])
    
    def predict(self, X):
        if len(X.shape)==1:                                                            
            X=X.reshape([len(X),1])
        X=(X-self.Xcenter)/self.Xscale
        self.forward(X)
        return self.Z[self.n_M]
    
    @staticmethod
    def R2(Y,Y_hat):
        return 1 - ANN.OLS(Y,Y_hat)/ANN.OLS(Y,Y.mean())
    
    @staticmethod
    def OLS(Y,Y_hat):
        return np.sum((Y-Y_hat)**2)
    

    @staticmethod
    def sigmoid(H):
        return 1/(1+np.exp(-H))
    
    @staticmethod
    def d_sigmoid(Z):
        return Z * (1 - Z)

    @staticmethod
    def d_tanh(Z):
        return 1 - Z*Z
    
    @staticmethod
    def ReLU(H):
        return H * (H>0)
    
    @staticmethod
    def d_ReLU(Z):
        return Z>0
    
    
    